In [ ]:
---
title: Impulse responses of spring-mass-damper system
description: Finite and infinite impulses
author: Daning H.
show-code: False
show-prompt: False
params:
    w0:
        input: numeric
        label: Natural frequency, omega_0
        value: 2.0
        step: 0.2
    zt:
        input: numeric
        label: Damping ratio, zeta
        value: 1.5
        step: 0.1
---

We visualize the response of a spring-mass-damper system, examine the components of the response, and connect these components to the concepts of forced response and initial transients in Laplace Transform.

The governing equation of the system is,
$$
y'' + 2\zeta\omega_0 y' + \omega_0^2 y = \frac{1}{w}(u(t-1)-u(t-1-w),\ y(0)=0,\ y'(0)=0
$$
where $\omega_0$ is the natural frequency, $\zeta$ is damping ratio, $y$ is the displacement, and the right-hand side is a finite impulse with width $w$.

The Laplace Transform of the output is
$$
Y(s) = \frac{1}{s^2 + 2\zeta\omega_0 s + \omega_0^2}\frac{\exp(-s)-\exp(-(1+w)s)}{ws}
$$
where the two terms are the forced response and initial transients, respectively.

If a Dirac delta function were used instead, then the RHS would be $\delta(t-1)$ and the Laplace Transform of the output is
$$
Y(s) = \frac{1}{s^2 + 2\zeta\omega_0 s + \omega_0^2}\exp(-s)
$$

Below, sweep over different widths of finite impulses and see when the two responses becomes almost the same.  How the resemblance change with respect to the system damping and frequency?

In [10]:
w0 = 2.0
zt = 1.5

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import sympy as sp

# ODE parameters
a  = 2*zt*w0
b  = w0**2

# Simulation setup
tf = 6.0
Nt = 101
Ns = 11
ts = np.linspace(0, tf, Nt)
s, t = sp.symbols('s t')

# t-domain results
inp = []
res = []
ss = np.logspace(-2, 0, Ns, endpoint=True)
for _s in ss:
    Ys = (sp.exp(-s) - sp.exp(-(1+_s)*s))/((s**2+a*s+b)*s)/_s
    yt = sp.inverse_laplace_transform(Ys, s, t)
    func = sp.utilities.lambdify(t, yt, 'numpy')

    inp.append([
        [0,1,1,1+_s,1+_s,tf],
        [0,0,1/_s,1/_s,0,0]])
    res.append(func(ts).real)

Ys = sp.exp(-s)/(s**2+a*s+b)
yt = sp.inverse_laplace_transform(Ys, s, t)
func = sp.utilities.lambdify(t, yt, 'numpy')
ref = func(ts).real

fig = make_subplots(rows=2, cols=1,
                    vertical_spacing=0.1,
                    subplot_titles=("Input", "Output"))

# Variable plots
## Input
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=inp[_i][0], y=inp[_i][1]),
        row=1, col=1)

## Output
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=res[_i]),
        row=2, col=1)

## Dirac delta
fig.add_trace(go.Scatter(
    visible=False, mode='lines', showlegend=False,
    line=dict(color="red", width=1, dash='dash'),
    x=[0,1,1,1,tf], y=[0,0,10,0,0]),
    row=1, col=1)

fig.add_trace(go.Scatter(
    visible=False, mode='lines', showlegend=False,
    line=dict(color="red", width=1, dash='dash'),
    x=ts, y=ref),
    row=2, col=1)

## Initial visibility
for _i in range(2):
    fig.data[_i*Ns].visible = True
fig.data[-1].visible = True
fig.data[-2].visible = True

# Create and add slider
steps = []
for _i in range(Ns):
    mask = [False] * (Ns*2+2)
    mask[_i] = True
    mask[_i+Ns] = True
    mask[-1] = True
    mask[-2] = True
    step = dict(
        method="update",
        args=[{"visible": mask}],
        label=f"{ss[_i]:3.2f}"
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "width = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_xaxes(title_text='', range=[0,tf], row=1, col=1)
fig.update_yaxes(title_text='', range=[-0.1,10.], row=1, col=1)
fig.update_xaxes(title_text='', range=[0,tf], row=2, col=1)
fig.update_yaxes(title_text='', range=[np.min(res), np.max(res)], row=2, col=1)

fig.update_layout(
    sliders=sliders,
    autosize=False,
    width=800,
    height=600)

fig.show()